In [2]:
%matplotlib notebook
import os
import json
import time
import pickle
import requests
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt
from table_reader import TableReader
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

Load Data

In [3]:
tr = TableReader()
df = tr.properties_vector(include_amenitites=True)
tr.close()

In [4]:
features = df[df.columns.drop(['price', 'listingID'])]
label = df['price']

df.describe()

,listingID,accomodates,bathrooms,bedrooms,beds,price,Apartment,Condominium,Guest suite,House,...,Family/kid friendly,Host greets you,Paid parking off premises,Laptop friendly workspace,Bathtub,translation missing: en.hosting_amenity_50,Private entrance,Kitchen,Oven,Gym
count,1.125100e+04,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,...,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.000000,11251.00000
mean,1.864017e+07,3.380322,1.271887,1.268332,1.790685,127.482802,0.458537,0.093325,0.047374,0.196338,...,0.409564,0.015465,0.092436,0.407608,0.021154,0.034308,0.105413,0.913252,0.013154,0.15252
std,9.446281e+06,1.921102,0.544124,0.810317,1.186819,75.388296,0.498300,0.290900,0.212446,0.397245,...,0.491775,0.123400,0.289654,0.491411,0.143903,0.182027,0.307098,0.281478,0.113941,0.35954
min,3.344000e+03,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.185777e+07,2.000000,1.000000,1.000000,1.000000,75.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000
50%,1.901856e+07,3.000000,1.000000,1.000000,1.000000,103.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000
75%,2.646554e+07,4.000000,1.500000,2.000000,2.000000,159.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000
max,3.488810e+07,12.000000,5.000000,6.000000,9.000000,399.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


Split data to training data and test data

In [5]:
X_train, X_test, y_train, y_test = tts(features, label, test_size=0.2)


Simple Regression with Ordinary Least Squares (OLS)

In [6]:
regr = LinearRegression()
regr.fit(X_train,y_train)
print(mean_squared_error(y_test, regr.predict(X_test)))
print(regr.score(X_test,y_test))

3086.270686579619
0.44613124170029606


Ridge Regression

In [7]:
clf = Ridge(alpha=0.5)
clf.fit(X_train, y_train)
print(mean_squared_error(y_test, clf.predict(X_test)))
print(clf.score(X_test, y_test))

3086.302554656707
0.44612552258681626


Choose alpha for Ridge Regression

In [8]:
import numpy as np

# try 200 different alphas between -10 and -2
n_alphas = 200
alphas = np.logspace(-10, -2, n_alphas)

clf = RidgeCV(alphas=alphas)
clf.fit(X_train, y_train)

#which alpha did it pick?
print(clf.alpha_)

0.01


In [9]:
clf.score(X_test, y_test)

0.4461148013834957

Lasso Regression

In [10]:
clf = Lasso(alpha=0.5)
clf.fit(X_train, y_train)
print(mean_squared_error(y_test, clf.predict(X_test)))
clf.score(X_test, y_test)

3154.6954461107916


0.43385158756519904

Logistic Regression

In [11]:
model = LogisticRegression().fit(X_train, y_train)
get_internal_params(model)


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


NameError: name 'get_internal_params' is not defined

In [ ]:
print(mean_squared_error(y_test, clf.predict(X_test)))
clf.score(X_test, y_test)